### Connecting to drive



In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL/HW2


Mounted at /gdrive
/gdrive/My Drive/AN2DL/HW2


### Importing libraries and fixing randomness



In [3]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [4]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.14.0


In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [ ]:
%pip install ipympl
%matplotlib widget

### Load and process data

In [ ]:
datas = np.load('training_dataset/training_data.npy', allow_pickle=True)
indexes = np.load('training_dataset/valid_periods.npy', allow_pickle=True)
labels = np.load('training_dataset/categories.npy', allow_pickle=True)
# Create a pandas DataFrame
data_dict = {'indexes': indexes.tolist(), 'data': datas.tolist(), 'labels': labels.tolist()}
df = pd.DataFrame(data_dict)

# Display the DataFrame
df.head()

# df.dropna(axis=0, how='any', inplace=True)

In [7]:
print(df.shape)
df.head(10)

(48000, 3)


,data,indexes,labels
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2325, 2776]",D
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2325, 2776]",D
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2325, 2776]",D
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2712, 2776]",D
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2682, 2776]",D
5,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2128, 2776]",D
6,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2682, 2776]",D
7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2682, 2776]",D
8,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1972, 2776]",D
9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2428, 2776]",D


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48000 entries, 0 to 47999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   indexes  48000 non-null  object
 1   data     48000 non-null  object
 2   labels   48000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [15]:
category = 'A'
sample = np.random.randint(1000)
window = 25

label = labels == category

data = datas[label][sample]
index = indexes[label][sample]

start = index[0]
end = index[1]


In [19]:
indexes[labels == 'A']

array([[2536, 2776],
       [2608, 2776],
       [2608, 2776],
       ...,
       [2614, 2776],
       [2615, 2776],
       [2676, 2776]])